In [ ]:
#This code will update a PDB file by changing atom names, while preserving PDB format. 
#Example use case: The commercially available unnatural amino acid thiazolylalanine, is incorporated into 
#a peptide, by first building the peptide with histidine as a starting off point to build upon (in Chimera, PyMOL, 
#Maestro, etc --pick your favorite softwar to build peptides). Then, we want to replace the histidine atom names
#with the atom names you want (usually for purposes that another software can recognize your unnatural amino acid).


In [ ]:
import numpy as np
import os
import glob
from Bio.PDB import PDBIO
import sys
from Bio.PDB import PDBIO
from Bio.PDB.PDBParser import PDBParser
io = PDBIO()

In [ ]:
#Path to already built, initial PDB files
Path='/Users/yourpath'

#good to see how many PDB files you want to change at once
pdbfiles = []
for file in glob.glob("*.pdb"):
    pdbfiles.append(file)
    
print(len(pdbfiles))

In [ ]:
#Think about the "rules" you want. For example, to change the atoms in my modified histidine to thiazolylalanine, 
#1) replace "HIS" with "TZA", where TZA is my 3-lettter code for thiazolylalanine
#2) replace "CG" with "C2"
#3) replace "CD" with "C4"
#4) replace "CE" with "C3"
#5) replace "SX" with "SS"
#6) replace last colun "N1+" with N1


In [ ]:
for file in pdbfiles:
    #for file, get the header and tail (usually PDB files have ATOM and TER entries)
    with open(file) as f:
        head_lines=[]
        tail_lines=[]
        lines = f.readlines()
        tail=0
        head=0
        for line in lines:
            if 'ATOM' not in line and 'TER' not in line and head==0:
                head_lines.append(line)
            if tail ==1:
                tail_lines.append(line)
            if 'TER' in line:
                tail_lines.append(line)
                tail=1
                head=1
    f.close()
    #load the pdb parser
    PDB_input = file
    parser = PDBParser()
    structure = parser.get_structure('self', PDB_input)
    
    first_residue = 0
    encounter_n = 0
    for model in structure:
        for chain in model:
            for residue in chain:
                first_residue+=1
                for atom in residue:
                    if 0 < encounter_n < 5:
                        if atom.name.startswith('C') and len(atom.name)>2:
                            atom.fullname = ' C' + str(encounter_n)
                        encounter_n += 1
                    if residue.resname == 'HIS':
                        if atom.name.startswith('S') and len(atom.name)>2:
                            atom.fullname = ' S1'
                        if atom.name.startswith('CD') and len(atom.name)>2:
                            atom.fullname = ' C4'
                        if atom.name.startswith('CE') and len(atom.name)>2:
                            atom.fullname = ' C3'
                        if atom.name.startswith('CG') and len(atom.name)>=2:
                            atom.fullname = ' C2'
                        if atom.name.startswith('ND') and len(atom.name)>2:
                            atom.fullname = ' N2'
                encounter_n = 0  
    #saving
    io.set_structure(structure)
    io.save(Path+file[:-4]+'_middle.pdb')
    
    with open(Path+file[:-4]+'_middle.pdb','r') as contents:
      save = contents.read()
    with open(Path+file[:-4]+'_final.pdb','w') as contents:
        for line in head_lines:
            contents.write(line)
        contents.write(save)
        for line in tail_lines:
            contents.write(line)
    os.remove(Path+file[:-4]+'_middle.pdb')

In [ ]:
#lastly, run these two SED commands on the terminal tab that has your saved, modified pdbs in to rename residues

sed -i '' 's/HIS/TZA/g' *_final.pdb
sed -i '' 's/N1+/N1/g' *_final.pdb
